In [ ]:
import numpy
import scipy.stats
%matplotlib widget
from matplotlib import pyplot

In [ ]:
# Set up a grid for wet mass
w_min, w_max = 0.001, 1e6
w_grid = 10.**numpy.linspace(numpy.log10(w_min), numpy.log10(w_max), 1000)

## Prey preference [(Scott et al. 2014)](https://doi.org/10.1111/2041-210X.12256)
* for a single species: $\sigma = 1$
* for a community: $\sigma = 2$ or $\sigma = \ln{10} \approx 2.3$

In [ ]:
beta = 100.                        # preferred predator : prey amss ratio
sigmas = [1, 2., numpy.log(10.)]   # standard deviation of sleecivity function (ln g)
w_predator = 100.
fig, ax = pyplot.subplots()
for sigma in sigmas:
    rho = numpy.exp(-(numpy.log(w_grid) - numpy.log(w_predator / beta))**2 / (2. * sigma**2))
    ax.plot(w_grid, rho, label='$\sigma = %.2f$' % sigma)
ax.set_xscale('log')
ax.axvline(w_predator, linestyle='--', color='k', label='predator mass')
ax.set_ylabel('predator preference (-)')
ax.set_xlabel('prey mass (g)')
ax.grid()
ax.legend();

## Clearance rate

In [ ]:
benoit_blanchard = 640 * w_grid**0.82
acuna = 10.**14.076 * w_grid**0.926 * 1e-3 * 365 * numpy.exp(-0.65 / 8.62e-5 / (273.15 + 13))  # convert from L to m3, from 1/d to 1/yr, apply Arrhenius temperature correction
acuna2 = 91.32 * w_grid**0.776
kiorboe = 10.**7.31 * (0.1 * w_grid)**(1.01) * 1e-6 * 365 * 2.8**-0.2  # convert from ml to m3, from 1/d to 1/yr, apply Q10 temperature correction

fig, ax = pyplot.subplots()

ax.loglog(w_grid, benoit_blanchard, '-', label='Benoît et al. 2004, Blanchard et al. 2009')
ax.loglog(w_grid[w_grid<1e4], acuna[w_grid<1e4], '-', label='Acuña et al. 2009')
ax.loglog(w_grid[w_grid<10.], kiorboe[w_grid<10.], '-', label='Kiørboe et al. 2011')
ax.set_xlabel('individual mass (g)')
ax.set_ylabel('clearance rate (m³/yr)')
ax.axhline(365., linestyle='--', color='k', label='1 m³/d')
ax.legend()
ax.grid()

import csv
ww, clearance = [], []
with open('../datasets/Acuna_et_al_2012_table_S1.dat') as f:
    for name2value in csv.DictReader(f, delimiter='\t'):
        if name2value['functional type'] in ('SF',):
            temp_corr = numpy.exp(-0.65 / 8.62e-5 * (1./ (273.15 + 13.) - 1./ (273.15 + float(name2value['Temp']))))
            ww.append(float(name2value['WW']))
            clearance.append(float(name2value['Clearance']) * temp_corr * 1e-3 * 365)
ax.loglog(ww, clearance, '.', mec='gray', mfc='w', label='Acuña et al. 2009 Fig2A', zorder=-1);

## Fisheries mortality and its size selectivity
* linear: [Blanchard et al, 2009](https://doi.org/10.1111/j.1365-2656.2008.01466.x)
* knife-edge: [Blanchard et al, 2012](https://doi.org/10.1098/rstb.2012.0231)
* sigmoid: [Blanchard et al, 2014](https://doi.org/10.1111/1365-2664.12238)

In [ ]:
fig, (ax1, ax2, ax3) = pyplot.subplots(ncols=3, figsize=(12,5))

# Linear
f = 0.09 * numpy.log10(w_grid) + 0.04; f[w_grid < 10.] = 0
ax1.semilogx(w_grid, f,'-')
ax1.set_title('linear')

# Knife-edge
f1 = numpy.full_like(w_grid, 0.8); f1[w_grid < 1.25] = 0
f2 = numpy.full_like(w_grid, 0.2); f2[w_grid < 1.25] = 0
ax2.semilogx(w_grid, f1,'-')
ax2.semilogx(w_grid, f2,'--')
ax2.set_title('knife-edge')

# Sigmoid
w25 = numpy.log(10.)   # wet mass where fishing pressure is 25% of maximum
w50 = numpy.log(100.)  # wet mass where fishing pressure is 50% of maximum
S2 = numpy.log(3)/(w50-w25)
S1 = S2 * w50
f = 0.8 * 1. / (1. + numpy.exp(S1 - S2 * numpy.log(w_grid)))
ax3.semilogx(w_grid, f, '-')
ax3.set_title('sigmoid')

ax1.set_ylabel('fisheries mortality (1/yr)')
for ax in (ax1, ax2, ax3):
    ax.set_xlabel('individual wet mass (g)')
    ax.set_xlim(.001, 1e6)
    ax.set_ylim(None, 1)
    ax.grid()

## Temperature dependence introduced by [Blanchard et al. (2012)](https://doi.org/10.1098/rstb.2012.0231)

In [ ]:
# Illustrate temperature dependence introduced by Blanchard et al. (2012, 10.1098/rstb.2012.0231)
# For comparison add curve for Q10=2
E_a = 0.63   # activation energy (eV)
T_ref = 13.  # reference temperature
k = 8.62e-5  # Boltzmann constant (eV K-1)
c1 = E_a / k / (T_ref + 273.15) # minus natural log of scale factor at reference temperature (equal to c1 in Blanchard et al.)
print('c1 = %.2f (cf. Blanchard et al. 2012)' % c1)
T = numpy.linspace(-2., 30, 1000)
tau = numpy.exp(c1 - E_a / k / (T + 273.15))
fig, ax = pyplot.subplots()
ax.plot(T, tau, '-', label='Arrhenius with $E_a = 0.63$ (Blanchard et al. 2012)')
ax.plot(T, 2.8**((T - T_ref) / 10.), '--', label='$Q_{10}=2.8$ (Kiørboe 2011, Hansen et al. 1997)')
ax.plot(T, 2.**((T - T_ref) / 10.), '--', label='$Q_{10}=2$')
ax.set_xlabel('temperature (degrees Celsius)')
ax.set_ylabel('relative rate (-)')
ax.axvline(T_ref, linestyle=':', color='k')
ax.legend()
ax.grid()